# HOW TO WRITE MACROS & MACRO USE CASES
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

#  HOW TO WRITE MACROS - PRACTICAL

# Macro function vs. macro class

## Function
* For programmers with background on a functional language (e.g. Fortran, Matlab, SPEC)
* Sequencial procedures to run an experiment

## Class
* For proammers with background on object-oriented languages (e.g. Java, C++, C#)
* Overriding an existing macros (e.g. inherit from `mv` macro and extend its funcionality)
* Some advanced features require use of classes e.g. define hook places or mandatory environment variables

# Macro plugins discovery
* Sardana built-in macros are importable from `sardana.macroserver.macros`
* The plugin discovery system is based on directory scanning and python module inspection
* Custom macros should be installed in one of the `MacroPath` directories:
 * Create /macros directory: `mkdir /macros`
 * In spock: `_MACRO_SERVER.put_property({"MacroPath":["/macros", "/usr/local/lib/python2.7/dist-packages/sardana/macroserver/macros/examples"]})`
 * Restart the Sardana server
* The path order is important! Macros in the higher position paths will take precedence over the lower position paths.

# Hello world

In [ ]:
from sardana.macroserver.macro import macro

@macro()
def hello_world(self):
    """This is an hello world macro"""
    self.print("Hello, World!")

# Using external python libraries in macros

* external libraries must be available for the python interpreter running the MacroServer server
* this can be configured using `PythonPath` property of MacroServer or `PYTHONPATH` OS environment variable
* external libraries can be reloaded at the MacroServer runtime with `rellib` macro

# Macro development tools

* `edmac <macro_name> [<module_name>]`
* `relmac`, `relmaclib`, `addmaclib`, `rellib`
* `www`, `post_mortem`
* `debug on|off`

## Acquire the active measurement group while oscilating a motor

### Acquire the active measurement group while oscilating a motor
* Read the documentation on [how to write macros](http://www.sardana-controls.org/en/latest/devel/howto_macros/index.html) before starting with this example
* Open the template: `kwrite sardana-training/macros/templates/experiment.py`
* Implement the macro according to the comments
* Deploy the macro in the `/macros` directory: `ln -s /sardana-training/macros/templates/experiment.py /macros`

## Perform a region scan

### Perform a region scan
* Configure the experiment using the expconf
 * `configure the showscan online`
* `prdef regscan`
* `regscan mot01 0.1 0 10 10 20 2`

#  MACRO FEATURES - EXAMPLES & REAL USE CASES

## MACRO FEATURES - EXAMPLES AND REAL USE CASES

### * Parameter Repetitions: BL09-Mistral ManyTomos collect macro

### * Custom Recorders: Dumb Recorder

### * Hooks: BL24-CIRCE CarbonScan macro

### * Access To The Scan Data: BL04-MSPD mythen_take macro

# PARAMETER REPETITIONS

## ManyTomos: Collect Macro at BL09-MISTRAL Beamline

## PARAMETER REPETITIONS
### Collect Macro ManyTomos at BL09-Mistral Beamline


* **BL09-Mistral**  is dedicated to **Tomography and Spectroscopy Imaging**


* **X-Ray Microscope** accepting scripts for performing the **data collection**


* Long scripts are needed for collecting the tomography image projections:  
Moving the **Sample Position**, the **ZonePlate Microscope Lens**, the **angles**,  
the **exposure times**, the **Energies**...


* Requirement: **macro** to create **X-Ray Microscope txt Scripts**


![TXM Microscope Scheme](res/TXM.jpg)


## PARAMETER REPETITIONS: 
### Collect Macro ManyTomos at BL09-Mistral


### MacroExecutor View of ManyTomos macro with a set of Parameters 

* ManyTomos generates a txt script, that is used by the BL09 TXM-Microscope to execute the data collection
* Example: 
 * Take Tomographies for two different energies
 
 ![Many Tomos Collect Macro](res/manytomos.jpg)
  

# CUSTOM RECORDERS 

## Dumb Recorder: A Sardana Custom Recorder Example

# CUSTOM RECORDERS: Dumb Recorder


* Sardana provides an **example of Custom Recorder**: the **DumbRecorder**

* DumbRecorder: Stores in a file the acquisitions done during a scan

* How to set up the Recorder:
    * From spock: Add the recorder path to the macroexecutor properties:  
    ```_MACRO_SERVER.put_property({"RecorderPath":["/usr/local/lib/python2.7/dist-packages/sardana-2.3.3a0-py2.7.egg/sardana/macroserver/recorders/examples"]})```
    * Restart the Sardana Server
    * Define the **Recorder**, the **Scan Directory** and the **Scan File**:
    ```
    senv ScanDir /tmp/
    senv ScanFile foo.txt
    senv ScanRecorder DumbRecorder
    ```
* Execute a scan from spock: 
    ```
    dscan mot01 0 2 1 0.1
    ```
* Results: Scan Output in ```/tmp/foo.txt``` file
  

# HOOKS

## CarbonScan: a Scan Macro using the Hooks at BL24-CIRCE

## HOOKS
### CarbonScan: a Scan Macro using the Hooks at BL24-CIRCE Beamline


* **Experiment:**
    * Experiment at BL24-CIRCE ALBA Beamline
    * PEEM Experiment: Photo Emission Electron Microscopy
    * Detector: PCO 2D Camera
    * Sample: **Graphene Carbon Monolayer**
    * Macro **CarbonScan**: Scan acquiring a 2D image at each Energy

* **Code:**

    * https://sourceforge.net/p/sardana/macros.git/ci/master/tree/ALBA_BL24_CIRCE/carbon_scan.py
    * https://sourceforge.net/p/sardana/macros.git/ci/master/tree/ALBA_BL24_CIRCE/peem_macros.py
  

## HOOKS
### CarbonScan: a Scan Macro using the Hooks at BL24-CIRCE Beamline

* **Requirements and Constraints**
    * Macro **CarbonScan**: Scan acquiring a 2D image at each Energy
    * Carbon beamline optics absorbs a lot of photons
        * Low flux arriving to the Carbon monoLayer Sample
        * Images normalized at the Carbon Edge (285eV) flux
    * PEEM 2D detector not still integrated in the Sardana

* **Solution:** Usage of a **Hook**
    * **CarbonScan** macro: Energy step scan for Graphene acquisition
    * ElectroMeter gets the flux at each energy for normalizing images
    * The CarbonScan macro calls the **Hook peemGetSingleImage**: 
        * Acquires PEEM images through a TangoDS at each point
        * Usage of LIMA Library in TangoDS to acquire the images
        * Sardana Scan hookplace **post-move**

## HOOKS
### CarbonScan: a Scan Macro using the Hooks at BL24-CIRCE Beamline

* **Resutls sample: 2D image at a given energy**
    * Low-energy electron microscopy (LEEM) image
    * 10x25um2 of Ruthenium (Ru) surface with 2 graphene flakes 
    * The graphene flakes thickness is of one single atomic C layer 

  ![Custom Recording](res/monolayer_carbon.jpg)

# DATA PROPERTY: Access To The Macro Data

## mythen_take: Mythen Data Collection Macro at BL04-MSPD

## DATA PROPERTY: Access To The Macro Data
### mythen_take: Mythen Data Collection macro at BL04-MSPD Beamline


* **Experiment:**
    * BL04-MSPD: Material Science and Powder Diffraction Beamline
    * XRD X-Ray Diffraction Experiment 
    * Batteries are studied during cycles of charge and discharge
    * Mythen 1D detector for collecting the spectrums
    * Potentiostat is setting and reading the Voltage applied to the battery
    * Potentiostat connected to Adlink data acquisition card
    * Results: 
        * Spectrum + Battery Voltage stored in the same file
        * Variation of the chemical species in the battery in function of the measure of charge/discharge

## DATA PROPERTY: Access To The Macro Data
### mythen_take: Mythen Data Collection macro at BL04-MSPD Beamline 

* Experiment Requirements:
    * Process the **voltage** read thanks to the Adlink DAQ
    * Store each voltage with the corresponding spectrum


* Solution: 
    * **mythen_take macro**: https://sourceforge.net/p/sardana/macros.git/ci/master/tree/ALBA_BL04_MSPD/mythen.py#l2210
    * mythen_take macro allows to: 
        * call a **custom ct** macro to measure the **Potentiostat Voltage**
        * Get the ct voltage data using the python property* **.data** *from the macro object
        * Perform a **Mythen spectrum acquisition**
        * Store the **Voltage** and the **Spectrum** in the same file  